In [ ]:
import json
import pickle
import plotly.express as px
import numpy as np

In [ ]:
# load location index
with open("../data/akl_loc_idx.pkl", 'rb') as f:
    loc_idx = pickle.load(f) # datazone to point index
    idx_loc = {v:k for k, v in loc_idx.items()} # point index to datazone    
    print(f" -- loaded location index with dimension {len(loc_idx)}")

# load time index
with open("../data/akl_t_idx.pkl", 'rb') as f:
    t_idx = pickle.load(f)
    print(f" -- loaded time index with dimension {len(t_idx)}")
    
# load precomputed odt
with open("../data/akl_odt.npy", 'rb') as f:
    odt = np.load(f)
    print(f" -- loaded odt cube with dimensions {odt.shape}")

In [ ]:
with open("../data/akl_polygons.geojson", 'r') as f:
    polys = json.load(f)

In [ ]:
# add id field to each feature (requried by plotly even though it's supposed to work via subproperty ref.)
for f in polys["features"]:
    f["id"] = f["properties"]["DZ2018"]        

In [ ]:
# remove properties DZ2018 as it is now duplicated
for f in polys["features"]:
    f["properties"] = f["properties"].pop("DZ2108", None)

In [ ]:
# filter polygons by those that have population centroids
pids = [k for k, v in loc_idx.items()]
valid_features = [f for f in polys["features"] if f["id"] in pids]
polys["features"] = valid_features
print("total features:", len(polys["features"]))

In [ ]:
# filter polygons by those that have valid journeys
jsum = np.sum(np.nansum(odt, axis=2), axis=0)
valid_idx = [i for i, v in enumerate(jsum) if v > 0]
valid_features = [f for f in polys["features"] if loc_idx[f["id"]] in valid_idx]
polys["features"] = valid_features
print(f"total features: {len(polys['features'])}")

In [ ]:
# combine polygons with duplicate feature ids
features = {}
for f in polys["features"]:
    fid = f["id"]
    if fid not in features:
        features[fid] = f        
    else:
        f_poly = f["geometry"]["coordinates"]
        features[fid]["geometry"]["coordinates"] += f_poly    
        #print(fid, len(features[fid]["geometry"]["coordinates"]))
polys["features"] = list(features.values())
print(f"total features: {len(polys['features'])}")

In [ ]:
# print combined features
for f in polys["features"]:
    npoly = len(f["geometry"]["coordinates"])
    if npoly > 1:
        print(f["id"], npoly)

In [ ]:
# optionally insert the odt data for each location
include_odt = True

if include_odt:   
    fmt = np.vectorize(lambda x: f"{x:.1f}")
    polys_dt = polys.copy()
    for f in polys_dt["features"]:
        # origin id and odt index of origin
        o = f["id"]
        o_idx = loc_idx[o]
        # get dt slice for the origin
        dt_slice = odt[o_idx]
        # set the feature dt property to the slice data
        f["dt"] = fmt(dt_slice).astype(np.float).tolist()     

In [ ]:
idx = 300
if include_odt:
    print(polys_dt["features"][idx])
else:
    print(polys["features"][idx])    

In [ ]:
# write file
with open("../data/akl_polygons_id.geojson", "w") as f:
    json.dump(polys, f)

In [ ]:
# write dt appended file
if include_odt:
    with open("../data/akl_polygons_id_dt.geojson", "w") as f:
        json.dump(polys_dt, f)  

In [ ]:
locations = [f["id"] for f in polys["features"]]
fig = px.choropleth_mapbox(
        geojson=polys, 
        featureidkey="id",
        locations=locations,        
        center = {"lat": -36.8485, "lon": 174.7633},
        mapbox_style="carto-positron",
        zoom=12)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()